In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [2]:
import cv2
import pickle
import numpy as np
import os
from datetime import datetime
from keras.models import load_model
import csv


#Load the model 
model = load_model('vgg16.h5')


def preprocess_image(frame):
    
    processed_image = frame / 255.0  
    return processed_image


attendance = True
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

with open('attendance_log.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Date', 'Time', 'Prediction'])

logged_predictions = set()  

def log_attendance(predicted_class):
    if predicted_class not in logged_predictions:  
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        current_date = now.strftime("%d/%m/%Y")

        with open('attendance_log.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([current_date, current_time, predicted_class])

        logged_predictions.add(predicted_class)  

while attendance:
    ret, frame = cap.read()
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
     
        padding = 50
        face_center_x = x + w // 2
        face_center_y = y + h // 2
        roi = frame[max(0, face_center_y - h // 2 - padding):min(frame.shape[0], face_center_y + h // 2 + padding),
                    max(0, face_center_x - w // 2 - padding):min(frame.shape[1], face_center_x + w // 2 + padding)]

        resized_face = cv2.resize(roi, (200, 400))

        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        processed_image = resized_face / 255.0  
        prediction = model.predict(np.array([processed_image]))

        class_labels =  ['J004-Jay Ajmera','J011-Savali Chavan','J012-Snehee Cheeda','J015 -Farid Damania','J016-Aditya Das','J021- Heet Dhandukia','J023-Anish Gharat','J024-Suchetan Ghosh',
                        'J025-Monish Gosar','J031- Naiktik Jain','J037-Rudra Joshi','J056-Atharva Rode','J058-Jash Shah','J065- Kallind Soni','J066- Naman Upadhyay','J069- Ismail Wangde','J074- Mihir Shah']
        predicted_class = class_labels[np.argmax(prediction)]

        log_attendance(predicted_class)

        annotation = predicted_class
        cv2.putText(frame, annotation, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    frame_height, frame_width, _ = frame.shape
    frame = cv2.resize(frame, (int(frame_width * (800 / frame_height)), 400))
  

    cv2.imshow('Face Detection and Prediction', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        



1/1 [==============================] - 0s 470ms/step
